### Import Libraries

In [172]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import datasets

### Functions

In [173]:
def cleanDescriptions(productDescriptions):
    '''Clean product descriptions
    Remove all characters other than digits and letters
    Remove all unecessary characters and noise to the classification model'''

    product_descriptions = productDescriptions.str.upper()

    new_series = pd.Series()

    for row in product_descriptions:
        clean_row = re.sub('\/', ' ', row, 0, re.IGNORECASE)
        clean_row = re.sub('[^A-Z0-9\s]', '', clean_row, 0, re.IGNORECASE)

        clean_row = clean_row.str.strip()


        new_series = new_series.append(pd.Series([clean_row]),  ignore_index=True)
        
        
    return new_series.to_numpy()  

### Import Dataset

In [174]:
# Import Cisco’s Public Export Product Data (PEPD)
raw_data = pd.read_csv("Cisco_raw_data.csv", encoding='cp1252')

c:\Users\GJ584JM\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### ETL

In [175]:
# Remove SERVICE Product Family
raw_data = raw_data.loc[raw_data['Product Family'] != "SERVICE"] 

# Keep only necessary fields
raw_data = raw_data[["Product Desc","US ECCN"]] 

# Check and drop nas
print(raw_data.isna().sum()) 
raw_data.dropna(axis=0, inplace=True)

 # Remove both the leading and the trailing white spaces 
raw_data['Product Desc'] = raw_data['Product Desc'].str.strip()

# Count number of words in each description and keep only descriptions with more than 5 words
raw_data['Number of words'] = raw_data['Product Desc'].apply(lambda desc : len(desc.split())) 
raw_data = raw_data.loc[raw_data['Number of words'] > 5] 

# Label predictors 
raw_data["Predictor"] = raw_data['US ECCN'].apply(lambda eccn : 0 if eccn == "EAR99" else 1) 
raw_data["Predictor Name"] = raw_data['US ECCN'].apply(lambda eccn : "Not Controlled" if eccn == "EAR99" else "Controlled")

# Drop unecessary fields
raw_data.drop(['US ECCN', 'Number of words'], axis=1, inplace=True) 

# Remove duplicates
df = raw_data.drop_duplicates()

Product Desc    12
US ECCN          8
dtype: int64


### Reducing the data size

In [176]:
# Reducing significantly the data size to run tests faster before running the final model
df = df[0:2000]

### Train Test Split

In [177]:
# Check predictors frequency
print(df["Predictor Name"].value_counts(normalize=True))

# BAG OF WORDS (BOW)
X = df['Product Desc']
y = df['Predictor']

# Create training and test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, stratify=y, random_state=7)

Controlled        0.6645
Not Controlled    0.3355
Name: Predictor Name, dtype: float64


### Clean Data

In [178]:
print(x_train[0:10])
x_train2 = cleanDescriptions(x_train[0:10])
print("__\n",x_train2[0:10])

63                            NCS 2002 DC VERSION 2, ANSI
3275           ^Cisco GS 2520 LAN BASE WITH EXPRESS SETUP
1024                   CISCO 3200 SERIES IOS IP PLUS 3DES
1373    ^Cisco 7200 Series IOS SERVICE PROVIDER/SECURE...
2945                   CISCO 3745 SER IOS ENTERPRISE BASE
2408    ^Cisco 4500/4700 Series IOS IP PLUS IPSEC 3DES...
1085    ^Integrated Services Module for IPSec and MPPE...
198                     CISCO ISR 4451-X IOS XE UNIVERSAL
2900    ^Cisco 2801 IOS  ASK9-AESK9 FEAT SET FACTORY U...
121      CISCO 8200 32X400G QSFPDD 1RU FIXED SYSTEM W/HBM
Name: Product Desc, dtype: object
__
 ['NCS 2002 DC VERSION 2 ANSI' 'CISCO GS 2520 LAN BASE WITH EXPRESS SETUP'
 'CISCO 3200 SERIES IOS IP PLUS 3DES'
 'CISCO 7200 SERIES IOS SERVICE PROVIDER SECURED SHELL 3DES TITAN SKU REDUCTION'
 'CISCO 3745 SER IOS ENTERPRISE BASE'
 'CISCO 4500 4700 SERIES IOS IP PLUS IPSEC 3DES TITAN SKU REDUCTION'
 'INTEGRATED SERVICES MODULE FOR IPSEC AND MPPE ENCRYPTION'
 'CISCO ISR 4451X IOS 

C:\Users\GJ584JM\AppData\Local\Temp/ipykernel_3092/2143730139.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_series = pd.Series()
